In [ ]:
import numpy as np

In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms
from deepsense import neptune
import argparse
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [ ]:
ctx = neptune.Context()
ctx.tags.append('torch')

In [ ]:
parser = argparse.ArgumentParser(description='PyTorch CIF-10 Example')
parser.add_argument('--batch-size', type=int, default=64, metavar='N',
                    help='input batch size for training (default: 64)')
parser.add_argument('--epochs', type=int, default=10, metavar='N',
                    help='number of epochs to train (default: 10)')
parser.add_argument('--lr', type=float, default=0.01, metavar='LR',
                    help='learning rate (default: 0.01)')
parser.add_argument('--momentum', type=float, default=0.5, metavar='M',
                    help='SGD momentum (default: 0.5)')
parser.add_argument('--no-cuda', action='store_true', default=True,
                    help='disables CUDA training')
parser.add_argument('--seed', type=int, default=1, metavar='S',
                    help='random seed (default: 1)')
parser.add_argument('--log-interval', type=int, default=10, metavar='N',
                    help='how many batches to wait before logging training status')

args = parser.parse_args()
args.cuda = not args.no_cuda and torch.cuda.is_available()

In [ ]:
########################################################################
# The output of torchvision datasets are PILImage images of range [0, 1].
# We transform them to Tensors of normalized range [-1, 1].

transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=64,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

In [ ]:
# how to access shapes of the above?

In [ ]:
########################################################################
# 2. Define a Convolution Neural Network
# ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


model = Net()
# model.cuda() #uncomment 

In [ ]:
########################################################################
# 3. Define a Loss function and optimizer
# ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters())

In [ ]:
########################################################################
# 4. Train the network
# ^^^^^^^^^^^^^^^^^^^^
log_interval = 781

def train(epoch):
    train_loss = 0
    correct = 0
    
    model.train()
    for batch_idx, (data, target) in enumerate(trainloader):
        
        data, target = Variable(data), Variable(target)

        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()


        optimizer.zero_grad()
        output = model(data)
        train_loss += criterion(output, target).data[0] # sum up batch loss
        pred = output.data.max(1, keepdim=True)[1] # get the index of the max log-probability
        correct += pred.eq(target.data.view_as(pred)).long().cpu().sum()

        train_loss /= len(trainloader.dataset)
        if batch_idx % log_interval == 0 and batch_idx != 0:
            
            print('Train Epoch: {} Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(epoch,
                train_loss, correct, len(trainloader.dataset),
                100. * correct / len(trainloader.dataset)))

        ctx.channel_send('Log-loss training', epoch, train_loss)
        ctx.channel_send('Accuracy training', epoch, train_loss)


        
def test():
    model.eval()
    test_loss = 0
    correct = 0
    for data, target in testloader:
       
        data, target = Variable(data, volatile=True), Variable(target)
        output = model(data)
        test_loss += criterion(output, target).data[0] # sum up batch loss
        pred = output.data.max(1, keepdim=True)[1] # get the index of the max log-probability
        correct += pred.eq(target.data.view_as(pred)).long().cpu().sum()

    test_loss /= len(testloader.dataset)
    print('Test Epoch: {} Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(epoch,
        test_loss, correct, len(testloader.dataset),
        100. * correct / len(testloader.dataset)))
    
    ctx.channel_send('Log-loss testing', epoch, test_loss)
    ctx.channel_send('Accuracy testing', epoch, test_loss)


In [ ]:
epochs = 2 

In [ ]:
for epoch in range(1, epochs + 1):
    train(epoch)
    test()

In [5]:
print("-"*40)

----------------------------------------


In [4]:
#another try, with pytorch's ignite wrapper..

from __future__ import print_function
from argparse import ArgumentParser

from torch import nn
from torch.optim import SGD
from torch.utils.data import DataLoader
import torch
import torch.nn.functional as F
from torchvision.transforms import Compose, ToTensor, Normalize
from torchvision.datasets import MNIST

from ignite.engines import Events, create_supervised_trainer, create_supervised_evaluator
from ignite.metrics import CategoricalAccuracy, Loss


class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, 320)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return F.log_softmax(x)


def get_data_loaders(train_batch_size, val_batch_size):
    data_transform = Compose([ToTensor(), Normalize((0.1307,), (0.3081,))])

    train_loader = DataLoader(MNIST(download=True, root=".", transform=data_transform, train=True),
                              batch_size=train_batch_size, shuffle=True)

    val_loader = DataLoader(MNIST(download=False, root=".", transform=data_transform, train=False),
                            batch_size=val_batch_size, shuffle=False)
    return train_loader, val_loader


def run(train_batch_size, val_batch_size, epochs, lr, momentum, log_interval):
    cuda = torch.cuda.is_available()
    train_loader, val_loader = get_data_loaders(train_batch_size, val_batch_size)

    model = Net()
    if cuda:
        model = model.cuda()
    optimizer = SGD(model.parameters(), lr=lr, momentum=momentum)
    trainer = create_supervised_trainer(model, optimizer, F.nll_loss, cuda=cuda)
    evaluator = create_supervised_evaluator(model,
                                            metrics={'accuracy': CategoricalAccuracy(),
                                                     'nll': Loss(F.nll_loss)},
                                            cuda=cuda)

    @trainer.on(Events.ITERATION_COMPLETED)
    def log_training_loss(engine):
        iter = (engine.state.iteration - 1) % len(train_loader) + 1
        if iter % log_interval == 0:
            print("Epoch[{}] Iteration[{}/{}] Loss: {:.2f}"
                  "".format(engine.state.epoch, iter, len(train_loader), engine.state.output))

    @trainer.on(Events.EPOCH_COMPLETED)
    def log_validation_results(engine):
        evaluator.run(val_loader)
        metrics = evaluator.state.metrics
        avg_accuracy = metrics['accuracy']
        avg_nll = metrics['nll']
        print("Validation Results - Epoch: {}  Avg accuracy: {:.2f} Avg loss: {:.2f}"
              .format(engine.state.epoch, avg_accuracy, avg_nll))

    trainer.run(train_loader, max_epochs=epochs)
    


run(train_batch_size = 64,
    val_batch_size = 64,
    epochs = 10,
    lr = 0.01,
    momentum = 0.5,
    log_interval = 10)

Processing...
Done!


/Users/raf/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:32: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


Epoch[1] Iteration[10/938] Loss: 2.28
Epoch[1] Iteration[20/938] Loss: 2.30
Epoch[1] Iteration[30/938] Loss: 2.24
Epoch[1] Iteration[40/938] Loss: 2.25
Epoch[1] Iteration[50/938] Loss: 2.14
Epoch[1] Iteration[60/938] Loss: 2.15
Epoch[1] Iteration[70/938] Loss: 2.07
Epoch[1] Iteration[80/938] Loss: 2.06
Epoch[1] Iteration[90/938] Loss: 1.86
Epoch[1] Iteration[100/938] Loss: 1.92
Epoch[1] Iteration[110/938] Loss: 1.76
Epoch[1] Iteration[120/938] Loss: 1.67
Epoch[1] Iteration[130/938] Loss: 1.54
Epoch[1] Iteration[140/938] Loss: 1.42
Epoch[1] Iteration[150/938] Loss: 1.31
Epoch[1] Iteration[160/938] Loss: 1.17
Epoch[1] Iteration[170/938] Loss: 0.97
Epoch[1] Iteration[180/938] Loss: 1.12
Epoch[1] Iteration[190/938] Loss: 1.18
Epoch[1] Iteration[200/938] Loss: 1.10
Epoch[1] Iteration[210/938] Loss: 1.09
Epoch[1] Iteration[220/938] Loss: 1.07
Epoch[1] Iteration[230/938] Loss: 1.04
Epoch[1] Iteration[240/938] Loss: 0.88
Epoch[1] Iteration[250/938] Loss: 0.95
Epoch[1] Iteration[260/938] Loss: 

Epoch[3] Iteration[230/938] Loss: 0.20
Epoch[3] Iteration[240/938] Loss: 0.27
Epoch[3] Iteration[250/938] Loss: 0.24
Epoch[3] Iteration[260/938] Loss: 0.20
Epoch[3] Iteration[270/938] Loss: 0.37
Epoch[3] Iteration[280/938] Loss: 0.20
Epoch[3] Iteration[290/938] Loss: 0.28
Epoch[3] Iteration[300/938] Loss: 0.26
Epoch[3] Iteration[310/938] Loss: 0.32
Epoch[3] Iteration[320/938] Loss: 0.13
Epoch[3] Iteration[330/938] Loss: 0.42
Epoch[3] Iteration[340/938] Loss: 0.32
Epoch[3] Iteration[350/938] Loss: 0.41
Epoch[3] Iteration[360/938] Loss: 0.33
Epoch[3] Iteration[370/938] Loss: 0.34
Epoch[3] Iteration[380/938] Loss: 0.26
Epoch[3] Iteration[390/938] Loss: 0.30
Epoch[3] Iteration[400/938] Loss: 0.41
Epoch[3] Iteration[410/938] Loss: 0.14
Epoch[3] Iteration[420/938] Loss: 0.19
Epoch[3] Iteration[430/938] Loss: 0.30
Epoch[3] Iteration[440/938] Loss: 0.23
Epoch[3] Iteration[450/938] Loss: 0.34
Epoch[3] Iteration[460/938] Loss: 0.27
Epoch[3] Iteration[470/938] Loss: 0.19
Epoch[3] Iteration[480/93

Epoch[5] Iteration[450/938] Loss: 0.23
Epoch[5] Iteration[460/938] Loss: 0.17
Epoch[5] Iteration[470/938] Loss: 0.32
Epoch[5] Iteration[480/938] Loss: 0.10
Epoch[5] Iteration[490/938] Loss: 0.53
Epoch[5] Iteration[500/938] Loss: 0.23
Epoch[5] Iteration[510/938] Loss: 0.28
Epoch[5] Iteration[520/938] Loss: 0.32
Epoch[5] Iteration[530/938] Loss: 0.29
Epoch[5] Iteration[540/938] Loss: 0.33
Epoch[5] Iteration[550/938] Loss: 0.28
Epoch[5] Iteration[560/938] Loss: 0.18
Epoch[5] Iteration[570/938] Loss: 0.14
Epoch[5] Iteration[580/938] Loss: 0.36
Epoch[5] Iteration[590/938] Loss: 0.11
Epoch[5] Iteration[600/938] Loss: 0.18
Epoch[5] Iteration[610/938] Loss: 0.16
Epoch[5] Iteration[620/938] Loss: 0.21
Epoch[5] Iteration[630/938] Loss: 0.21
Epoch[5] Iteration[640/938] Loss: 0.26
Epoch[5] Iteration[650/938] Loss: 0.18
Epoch[5] Iteration[660/938] Loss: 0.21
Epoch[5] Iteration[670/938] Loss: 0.28
Epoch[5] Iteration[680/938] Loss: 0.34
Epoch[5] Iteration[690/938] Loss: 0.17
Epoch[5] Iteration[700/93

Epoch[7] Iteration[670/938] Loss: 0.18
Epoch[7] Iteration[680/938] Loss: 0.23
Epoch[7] Iteration[690/938] Loss: 0.30
Epoch[7] Iteration[700/938] Loss: 0.11
Epoch[7] Iteration[710/938] Loss: 0.28
Epoch[7] Iteration[720/938] Loss: 0.11
Epoch[7] Iteration[730/938] Loss: 0.18
Epoch[7] Iteration[740/938] Loss: 0.13
Epoch[7] Iteration[750/938] Loss: 0.34
Epoch[7] Iteration[760/938] Loss: 0.10
Epoch[7] Iteration[770/938] Loss: 0.27
Epoch[7] Iteration[780/938] Loss: 0.20
Epoch[7] Iteration[790/938] Loss: 0.14
Epoch[7] Iteration[800/938] Loss: 0.26
Epoch[7] Iteration[810/938] Loss: 0.16
Epoch[7] Iteration[820/938] Loss: 0.18
Epoch[7] Iteration[830/938] Loss: 0.15
Epoch[7] Iteration[840/938] Loss: 0.16
Epoch[7] Iteration[850/938] Loss: 0.45
Epoch[7] Iteration[860/938] Loss: 0.11
Epoch[7] Iteration[870/938] Loss: 0.17
Epoch[7] Iteration[880/938] Loss: 0.19
Epoch[7] Iteration[890/938] Loss: 0.10
Epoch[7] Iteration[900/938] Loss: 0.08
Epoch[7] Iteration[910/938] Loss: 0.13
Epoch[7] Iteration[920/93

Epoch[9] Iteration[890/938] Loss: 0.22
Epoch[9] Iteration[900/938] Loss: 0.10
Epoch[9] Iteration[910/938] Loss: 0.16
Epoch[9] Iteration[920/938] Loss: 0.27
Epoch[9] Iteration[930/938] Loss: 0.29
Validation Results - Epoch: 9  Avg accuracy: 0.98 Avg loss: 0.06
Epoch[10] Iteration[10/938] Loss: 0.18
Epoch[10] Iteration[20/938] Loss: 0.09
Epoch[10] Iteration[30/938] Loss: 0.05
Epoch[10] Iteration[40/938] Loss: 0.16
Epoch[10] Iteration[50/938] Loss: 0.10
Epoch[10] Iteration[60/938] Loss: 0.35
Epoch[10] Iteration[70/938] Loss: 0.27
Epoch[10] Iteration[80/938] Loss: 0.18
Epoch[10] Iteration[90/938] Loss: 0.12
Epoch[10] Iteration[100/938] Loss: 0.20
Epoch[10] Iteration[110/938] Loss: 0.15
Epoch[10] Iteration[120/938] Loss: 0.17
Epoch[10] Iteration[130/938] Loss: 0.19
Epoch[10] Iteration[140/938] Loss: 0.14
Epoch[10] Iteration[150/938] Loss: 0.60
Epoch[10] Iteration[160/938] Loss: 0.32
Epoch[10] Iteration[170/938] Loss: 0.33
Epoch[10] Iteration[180/938] Loss: 0.20
Epoch[10] Iteration[190/938] 